In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
from collections import OrderedDict
from functools import partial
from random import shuffle
import matplotlib.pyplot as plt
import os
import re
from keras.models import Model, load_model
from tensorflow.keras import Model, Input, regularizers, utils
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Flatten

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from tqdm import tqdm

In [3]:
Input_img = Input(shape=(224, 224, 3))  
    
#Common Discrimintaor
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)

x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)

x2 = MaxPool2D( (2, 2))(x2)

x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

x4 = Conv2D(32, (3, 3), activation='relu', padding='same')(x3)

x4 = MaxPool2D( (2, 2))(x4)

x5 = Conv2D(16, (3, 3), activation='relu', padding='same')(x4)

x6 = Conv2D(8, (3, 3), activation='relu', padding='same')(x5)

x6 = MaxPool2D((2, 2))(x6)

x7 = Conv2D(4, (3, 3), activation='relu', padding='same')(x6)

x8 = Conv2D(2, (3, 3), activation="relu", padding="same")(x7)

x8 = Flatten()(x8)

x9 = Dense(256, activation="relu")(x8)

x10 = Dense(64, activation="relu")(x9)

x11 = Dense(16, activation="relu")(x10)

x12 = Dense(4, activation="relu")(x11)

x13 = Dense(1, activation="sigmoid")(x12)

In [4]:
common_discriminator = Model(Input_img, x13)
common_discriminator.compile(optimizer='adam', loss='mse')

In [5]:
common_discriminator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 256)     7168      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 128)     295040    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 128)    0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      73792     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 32)      18464     
                                                             

In [6]:
import glob
root = 'E:/Download/AniGAN-main/face2anime/human-faces/'
face_images = glob.glob(root + "*.*")

In [7]:
root = 'E:/Download/AniGAN-main/face2anime/anime-faces/'
anime_images = glob.glob(root + "*.*")

In [8]:
len(face_images)

100

In [9]:
all_images = []

for i in tqdm(face_images):
    img = utils.load_img(i, target_size=(224,224,3))
    img = utils.img_to_array(img)
    all_images.append(img)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 78.86it/s]


In [10]:
for i in tqdm(anime_images):
    img = utils.load_img(i, target_size=(224,224,3))
    img = utils.img_to_array(img)
    all_images.append(img)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 238.44it/s]


In [11]:
labels = [1.0]*len(face_images) + [0.0]*len(anime_images)
# label = 1 for human faces
# label = 0 for anime faces
labels = np.array(labels)
print(len(labels))
print(labels.ndim)
print(labels.shape)

200
1
(200,)


In [12]:
all_images = np.array(all_images)

In [13]:
print(all_images.ndim)
print(all_images.shape)
print(all_images[0].shape)

4
(200, 224, 224, 3)
(224, 224, 3)


In [18]:
outcome = common_discriminator.fit(all_images, labels,
            epochs=2,
            batch_size=1,
            shuffle=True)

Epoch 1/2
200/200 [==============================] - 148s 738ms/step - loss: 0.5000
Epoch 2/2
200/200 [==============================] - 152s 760ms/step - loss: 0.5000


In [19]:
predictions = common_discriminator.predict(all_images)

7/7 [==============================] - 30s 4s/step


In [21]:
# print(predictions)

In [ ]:
#Above one is not so good, hence trying out another one.

In [22]:
from keras.applications.vgg19 import VGG19
# model = VGG19()
model = VGG19(weights="imagenet",input_shape=(224,224,3))
print(model.summary())

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
model_new = Model(model.input, model.layers[12].output)

In [25]:
model_new.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [35]:
images = face_images+anime_images

In [36]:
all_images = []

for i in tqdm(images):
    img = utils.load_img(i, target_size=(224,224,3))
    img = utils.img_to_array(img)
    
#     print("Before - ", img.shape)
    img = np.expand_dims(img,axis=0)
#     print("After - ", img.shape)
    
    all_images.append(img)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:05<00:00, 35.63it/s]


In [37]:
def encode_image(img):
    feature_vector = model_new.predict(img)
    return feature_vector

In [38]:
start_time = time()
encodings =[]

for ix,img in enumerate(all_images):
#     print(img.shape)
    encodings.append(encode_image(img))
    print("Encoding in Progress Time step %d "%ix)

# encoding_human = model_new.predict(all_images_human)

end_time = time()
print("Total Time Taken :",end_time-start_time)

1/1 [==============================] - 0s 213ms/step
Encoding in Progress Time step 0 
1/1 [==============================] - 0s 112ms/step
Encoding in Progress Time step 1 
1/1 [==============================] - 0s 132ms/step
Encoding in Progress Time step 2 
1/1 [==============================] - 0s 144ms/step
Encoding in Progress Time step 3 
1/1 [==============================] - 0s 195ms/step
Encoding in Progress Time step 4 
1/1 [==============================] - 0s 162ms/step
Encoding in Progress Time step 5 
1/1 [==============================] - 0s 125ms/step
Encoding in Progress Time step 6 
1/1 [==============================] - 0s 166ms/step
Encoding in Progress Time step 7 
1/1 [==============================] - 0s 139ms/step
Encoding in Progress Time step 8 
1/1 [==============================] - 0s 243ms/step
Encoding in Progress Time step 9 
1/1 [==============================] - 0s 138ms/step
Encoding in Progress Time step 10 
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 170ms/step
Encoding in Progress Time step 93 
1/1 [==============================] - 0s 122ms/step
Encoding in Progress Time step 94 
1/1 [==============================] - 0s 123ms/step
Encoding in Progress Time step 95 
1/1 [==============================] - 0s 152ms/step
Encoding in Progress Time step 96 
1/1 [==============================] - 0s 161ms/step
Encoding in Progress Time step 97 
1/1 [==============================] - 0s 156ms/step
Encoding in Progress Time step 98 
1/1 [==============================] - 0s 179ms/step
Encoding in Progress Time step 99 
1/1 [==============================] - 0s 171ms/step
Encoding in Progress Time step 100 
1/1 [==============================] - 0s 132ms/step
Encoding in Progress Time step 101 
1/1 [==============================] - 0s 164ms/step
Encoding in Progress Time step 102 
1/1 [==============================] - 0s 202ms/step
Encoding in Progress Time step 103 
1/1 [=======================

1/1 [==============================] - 0s 194ms/step
Encoding in Progress Time step 185 
1/1 [==============================] - 0s 169ms/step
Encoding in Progress Time step 186 
1/1 [==============================] - 0s 167ms/step
Encoding in Progress Time step 187 
1/1 [==============================] - 0s 140ms/step
Encoding in Progress Time step 188 
1/1 [==============================] - 0s 161ms/step
Encoding in Progress Time step 189 
1/1 [==============================] - 0s 153ms/step
Encoding in Progress Time step 190 
1/1 [==============================] - 0s 149ms/step
Encoding in Progress Time step 191 
1/1 [==============================] - 0s 129ms/step
Encoding in Progress Time step 192 
1/1 [==============================] - 0s 129ms/step
Encoding in Progress Time step 193 
1/1 [==============================] - 0s 120ms/step
Encoding in Progress Time step 194 
1/1 [==============================] - 0s 124ms/step
Encoding in Progress Time step 195 
1/1 [================

In [39]:
!mkdir saved

A subdirectory or file saved already exists.


In [40]:
with open("saved/encodings.pkl","wb") as f:
    pickle.dump(encodings,f)